In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix, accuracy_score

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression


from sklearn.neighbors import KNeighborsClassifier

from sklearn.ensemble import RandomForestClassifier

from sklearn import svm
import sklearn.model_selection as model_selection
from sklearn.metrics import f1_score
from sklearn.svm import SVC
from sklearn.utils import shuffle

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import ExtraTreesRegressor
from sklearn import preprocessing


In [26]:
# df = pd.read_csv('./data/data_processed.csv', index_col=0)
# df = pd.read_csv('./data/data_processed_normalized.csv', index_col=0)
df = pd.read_csv('./data/raw_data_cleaned.csv')

In [3]:
df.columns

Index(['Food Name', 'Water (g)', 'Energy (kal)', 'Protein (g)', 'lipid (g)',
       'Carbohydrate (g)', 'Fiber (g)', 'Ash (g)', 'Ca (mg)', 'Fe (mg)',
       'Mg (mg)', 'P (mg)', 'K (mg)', 'Na (mg)', 'Zn (mg)', 'Se (µg)',
       'Cu (mg)', 'Mn (mg)', 'Vc (mg)', 'Thiamin (mg)', 'Riboflavin (mg)',
       'Niacin (mg)', 'B6 (mg)', 'Folate,DFE (µg)', 'B12 (µg)', 'Va,RAE (µg)',
       'Ve (mg)', 'saturated (g)', 'monounsaturated (g)',
       'polyunsaturated (g)', 'trans (g)', 'Cholesterol (mg)', 'Caffeine (mg)',
       'phenolics (mg)', 'pH', 'Plain Occurences', 'Cool Occurences',
       'Warm Occurences', 'Cold Occurences', 'Heavy Cold Occurences',
       'Heavy Warm Occurences', 'Hot Occurences', 'Heavy Hot Occurences',
       '% Plain', '% Cool', '% Warm', '% Cold', '% Heavy Cold', '% Heavy Warm',
       '% Hot', '% Heavy Hot', 'hot_cold_scale', 'Plain', 'Cold', 'Hot',
       'Mode'],
      dtype='object')

In [4]:
df.head()

,Food Name,Water (g),Energy (kal),Protein (g),lipid (g),Carbohydrate (g),Fiber (g),Ash (g),Ca (mg),Fe (mg),...,% Cold,% Heavy Cold,% Heavy Warm,% Hot,% Heavy Hot,hot_cold_scale,Plain,Cold,Hot,Mode
0,Alfalfa,0.903084,0.036199,0.092251,0.0070,0.029565,0.035714,0.012024,0.093645,0.028747,...,0.0,0.0,0.0,0.0,0.0,0.5,3,0,0,Plain
1,dolichos sinensis,0.902082,0.020362,0.040590,0.0030,0.074421,0.080827,0.007014,0.051839,0.008214,...,0.0,0.0,0.0,0.0,0.0,0.5,3,0,0,Plain
2,hazelnuts,0.053164,0.710407,0.275830,0.6075,0.170252,0.182331,0.041082,0.095318,0.048255,...,0.0,0.0,0.0,0.0,0.0,0.5,3,0,0,Plain
3,"Beans, kidney",0.117641,0.376697,0.435055,0.0083,0.611785,0.468045,0.009018,0.119565,0.084189,...,0.0,0.0,0.0,0.0,0.0,0.5,3,0,0,Plain
4,"Peanuts,",0.065078,0.641403,0.476015,0.4924,0.164441,0.159774,0.032064,0.076923,0.047023,...,0.0,0.0,0.0,0.0,0.0,0.5,3,0,0,Plain


TO DO label encoder, temp solution

df['hot_cold_scale'] = df['hot_cold_scale'] * 10
df['hot_cold_scale'] = df['hot_cold_scale'].astype(int)

### Data Process

In [5]:
def normalize(df):
    df.loc["Water (g)": "pH"] = (df.loc["Water (g)": "pH"]-df.loc["Water (g)": "pH"].mean())/df.loc["Water (g)": "pH"].std()
    
    name = df.iloc[:, 0:1]
    inputs = df.loc[:, 'Water (g)':'pH'].astype(float)
    outputs = df.loc[:, 'pH':].drop(columns='pH')

    # normalized_inputs = (inputs-inputs.mean())/inputs.std()
    normalized_inputs=(inputs-inputs.min())/(inputs.max()-inputs.min())
    
    df = pd.concat([name, normalized_inputs, outputs], axis=1)
    return df
    

In [6]:
def occurrences(df):
    df["% Plain"] = df["Plain Occurences"]/(df["Plain Occurences"] + df["Cool Occurences"] + df["Warm Occurences"] + df["Cold Occurences"] + df["Heavy Cold Occurences"] + df["Heavy Warm Occurences"] + df["Hot Occurences"] + df["Heavy Hot Occurences"])

    df["% Cool"] = df["Cool Occurences"]/(df["Plain Occurences"] + df["Cool Occurences"] + df["Warm Occurences"] + df["Cold Occurences"] + df["Heavy Cold Occurences"] + df["Heavy Warm Occurences"] + df["Hot Occurences"] + df["Heavy Hot Occurences"])

    df["% Warm"] = df["Warm Occurences"]/(df["Plain Occurences"] + df["Cool Occurences"] + df["Warm Occurences"] + df["Cold Occurences"] + df["Heavy Cold Occurences"] + df["Heavy Warm Occurences"] + df["Hot Occurences"] + df["Heavy Hot Occurences"])

    df["% Cold"] = df["Cold Occurences"]/(df["Plain Occurences"] + df["Cool Occurences"] + df["Warm Occurences"] + df["Cold Occurences"] + df["Heavy Cold Occurences"] + df["Heavy Warm Occurences"] + df["Hot Occurences"] + df["Heavy Hot Occurences"])

    df["% Heavy Cold"] = df["Heavy Cold Occurences"]/(df["Plain Occurences"] + df["Cool Occurences"] + df["Warm Occurences"] + df["Cold Occurences"] + df["Heavy Cold Occurences"] + df["Heavy Warm Occurences"] + df["Hot Occurences"] + df["Heavy Hot Occurences"])

    df["% Heavy Warm"] = df["Heavy Warm Occurences"]/(df["Plain Occurences"] + df["Cool Occurences"] + df["Warm Occurences"] + df["Cold Occurences"] + df["Heavy Cold Occurences"] + df["Heavy Warm Occurences"] + df["Hot Occurences"] + df["Heavy Hot Occurences"])

    df["% Hot"] = df["Hot Occurences"]/(df["Plain Occurences"] + df["Cool Occurences"] + df["Warm Occurences"] + df["Cold Occurences"] + df["Heavy Cold Occurences"] + df["Heavy Warm Occurences"] + df["Hot Occurences"] + df["Heavy Hot Occurences"])

    df["% Heavy Hot"] = df["Heavy Hot Occurences"]/(df["Plain Occurences"] + df["Cool Occurences"] + df["Warm Occurences"] + df["Cold Occurences"] + df["Heavy Cold Occurences"] + df["Heavy Warm Occurences"] + df["Hot Occurences"] + df["Heavy Hot Occurences"])

    return df

In [7]:
def linear_scale(df):
#     df["hot_cold_scale"] = (df["% Plain"] * (3.0/7.0)) + (df["% Cool"] * (2.0/7.0)) + (df["% Warm"] * (4.0/7.0)) + (df["% Cold"] * (1.0/7.0)) + (df["% Heavy Cold"] * (0.0/7.0)) + (df["% Heavy Warm"] * (5.0/7.0)) + (df["% Hot"] * (6.0/7.0)) + (df["% Heavy Hot"] * (7.0/7.0))
    df["hot_cold_scale"] = (df["% Plain"] * (3.0/6.0)) + (df["% Cool"] * (2.0/6.0)) + (df["% Warm"] * (4.0/6.0)) + (df["% Cold"] * (1.0/6.0)) + (df["% Heavy Cold"] * (0.0/6.0)) + (df["% Heavy Warm"] * (4.5/6.0)) + (df["% Hot"] * (5.0/6.0)) + (df["% Heavy Hot"] * (6.0/6.0))

    return df

In [8]:
def mode(df):
    df['Plain'] = df['Plain Occurences'].astype(int)
    df['Cold'] = df['Cool Occurences'].astype(int) + df['Cold Occurences'].astype(int) + df['Heavy Cold Occurences'].astype(int)
    df['Hot'] = df['Warm Occurences'].astype(int) + df['Heavy Warm Occurences'].astype(int) + df['Hot Occurences'].astype(int) + df['Heavy Hot Occurences'].astype(int)
    newDf = pd.DataFrame(df.loc[:, ['Plain', 'Cold', 'Hot']].idxmax(axis=1))
    df["Mode"] = newDf[0]
    return df

In [9]:
def nanValuesRid(df):
    not_too_many_null_cols = df.columns[df.isnull().mean() < 0.3]
    df = df[not_too_many_null_cols]
    
    return df

In [10]:
def iterative_imputation(df):
    ii_imp = IterativeImputer(estimator=ExtraTreesRegressor(), max_iter=10, random_state=1121218)

    name = df.iloc[:, 0:1]
    inputs = df.loc[:, 'Water (g)':'pH']
    outputs = df.loc[:, 'pH':].drop(columns='pH')
    
    inputs = ii_imp.fit_transform(inputs)
    new_inputs = pd.DataFrame(inputs, columns = not_too_many_null_cols[1:35]) 
    
    return pd.concat([name, new_inputs, outputs], axis=1)

In [11]:
def label_encoder(df):
    # reading word labels
    label_encoder = preprocessing.LabelEncoder()

    # encode word labels in column
    df['hot_cold_scale'] = label_encoder.fit_transform(df['hot_cold_scale'])

    df['hot_cold_scale'].unique()

    return df
    

In [12]:
def data_process(df):
    df = normalize(df)
    
    df = nanValuesRid(df)
    
    df = iterative_imputation(df)
    
    df = label_encoder(df)
    
    return df

In [32]:
df = occurrences(df)
df = linear_scale(df)
df = mode(df)
cols = list(df.columns.values)
cols.pop(cols.index('hot_cold_scale'))
df = df[cols+['hot_cold_scale']]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 241 entries, 0 to 240
Data columns (total 60 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             241 non-null    object 
 1   Water (g)              241 non-null    float64
 2   Energy (kal)           241 non-null    float64
 3   Protein (g)            239 non-null    float64
 4   lipid (g)              239 non-null    float64
 5   Carbohydrate (g)       240 non-null    float64
 6   Fiber (g)              221 non-null    float64
 7   Sugars (g)             168 non-null    float64
 8   Ash (g)                227 non-null    float64
 9   Ca (mg)                237 non-null    float64
 10  Fe (mg)                238 non-null    float64
 11  Mg (mg)                236 non-null    float64
 12  P (mg)                 238 non-null    float64
 13  K (mg)                 237 non-null    float64
 14  Na (mg)                238 non-null    float64
 15  Zn (mg

### Logistic Regression

In [35]:
def logistic_regression(df, y_col_name):
    """
    Apply sample logistic regression model to dataframe

    df : (dataframe)
    x : (compounds column as int)
    y : (classification column as int)
    """
    # values of each column
    x = df.iloc[:, 1:] # df.iloc[2:, 1:39]??
    y = df[y_col_name][:]

    # split data into training and test set
    # 75 training, 25 testing
    xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size = 0.2, random_state = 0)
    
    train = data_process(pd.concat([xtrain, ytrain]))
    xtrain = trian.iloc[:, 1:39]
    ytrain = train[y_col_name][:]
    
    test = data_process(pd.concat([xtest, ytest], axis=1))
    xtest = test.iloc[:, 1:39]
    ytest = test[y_col_name][:]


    # standardize and scale data
    scaler = StandardScaler()
    xtrain = scaler.fit_transform(xtrain)
    xtest = scaler.transform(xtest)

    # modeling + analysis
    model = LogisticRegression(random_state=0)
    model.fit(xtrain, ytrain.ravel())
    y_pred = model.predict(xtest)

    # confusion matrix of test size
    conf_m = confusion_matrix(ytest, y_pred)
    print("Confusion Matrix : ", conf_m)

    # accuracy score of test size
    print ("Accuracy : ", accuracy_score(ytest, y_pred))

In [36]:
logistic_regression(df, "hot_cold_scale")

KeyError: 'Water (g)'

### KNN

In [ ]:
def knn(df, y_col_name):
    x = df.iloc[:, 1:39]
    y = df[y_col_name][:]
    
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)
    knn = KNeighborsClassifier(n_neighbors = 5)
    knn.fit(X_train, y_train)

    y_pred = knn.predict(X_test)
    
    print ("Accuracy : ", accuracy_score(y_test, y_pred))

In [ ]:
knn(df, "hot_cold_scale")

more processing for following

In [ ]:
def label_mode(row):
    if row['Mode'] == 'Plain':
        return 0
    elif row['Mode'] == 'Cold':
        return 1
    elif row['Mode'] == 'Hot':
        return 2
    
df['mode_code'] = df.apply (lambda row: label_mode(row), axis=1)
df = df.drop(columns='Food Name')

### Random Forest

In [ ]:
def random_forest(df):
    X = df.drop(["Plain Occurences", "Cool Occurences", "Warm Occurences", "Cold Occurences", "Heavy Cold Occurences", "Heavy Warm Occurences","Hot Occurences", "Heavy Hot Occurences", "mode_code", "Mode"], axis = 1)
    Y = df["mode_code"].astype('int')
    factor = pd.factorize(df['Mode'])
    df.Mode = factor[0]
    definitions = factor[1]
    print(definitions)
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.25, random_state = 21)
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 42)
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    reversefactor = dict(zip(range(3),definitions))
    y_test = np.vectorize(reversefactor.get)(y_test)
    y_pred = np.vectorize(reversefactor.get)(y_pred)
    print(pd.crosstab(y_test, y_pred, rownames=['Actual Temp'], colnames=['Predicted Temp']))

In [ ]:
random_forest(df)

### SVM

In [ ]:
def svm(df):
    X = df.drop(["Plain Occurences", "Cool Occurences", "Warm Occurences", "Cold Occurences", "Heavy Cold Occurences", "Heavy Warm Occurences","Hot Occurences", "Heavy Hot Occurences", "mode_code", "Mode"], axis = 1)
    y = df["mode_code"]
    X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, train_size=0.80, test_size=0.20, random_state=101)
    rbf = SVC(kernel='rbf', gamma=0.5, C=0.1).fit(X_train, y_train)
    poly = SVC(kernel='poly', degree=3, C=1).fit(X_train, y_train)
    poly_pred = poly.predict(X_test)
    rbf_pred = rbf.predict(X_test)
    poly_accuracy = accuracy_score(y_test, poly_pred)
    poly_f1 = f1_score(y_test, poly_pred, average='weighted')
    print('Accuracy (Polynomial Kernel): ', "%.2f" % (poly_accuracy*100))
    print('F1 (Polynomial Kernel): ', "%.2f" % (poly_f1*100))
    rbf_accuracy = accuracy_score(y_test, rbf_pred)
    rbf_f1 = f1_score(y_test, rbf_pred, average='weighted')
    print('Accuracy (RBF Kernel): ', "%.2f" % (rbf_accuracy*100))
    print('F1 (RBF Kernel): ', "%.2f" % (rbf_f1*100))

In [ ]:
svm(df)